In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

# conectar base de datos
conn=sql.connect('Data/db_moviesi.db')
cur=conn.cursor()

# ver tablas disponibles en base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('full_ratings',),
 ('full_ratings_sin_dup',),
 ('reco',)]

# **2. Sistemas de recomendación contenido general**

In [2]:
df = pd.read_sql("SELECT * FROM full_ratings", conn)
df.head()

,user_id,movie_id,rating,movie_title,movie_genres,movie_clean_title,movie_year,fecha_nueva
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995),1995.0,2000-07-30
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995),1995.0,2000-07-30
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,Heat (1995),1995.0,2000-07-30
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven (a.k.a. Se7en) (1995),1995.0,2000-07-30
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The (1995)",1995.0,2000-07-30


In [3]:
# géneros
unique_genres = set()
for genres in df['movie_genres'].str.split('|'):
    unique_genres.update(genres)

# Contar la cantidad de géneros únicos
num_unique_genres = len(unique_genres)
print("Número de géneros diferentes:", num_unique_genres)

# Imprimir los géneros diferentes
print("Géneros diferentes:")
for genre in unique_genres:
    print(genre)

Número de géneros diferentes: 20
Géneros diferentes:
Adventure
IMAX
Children
Fantasy
Romance
Animation
Film-Noir
Horror
Drama
Crime
Western
Action
Mystery
(no genres listed)
Thriller
Sci-Fi
Musical
Comedy
Documentary
War


In [4]:
# Ajustar la configuración de visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

pregunta. vamos a poner todos en un solo notebook?, de lo contrario hacer la copia no sobraría?

In [5]:
df2 = df.copy()
# Convertir la variable de género en una lista de géneros para cada película
df2['movie_genres_list'] = df2['movie_genres'].str.split('|')

# Obtener todas las categorías únicas de géneros
unique_genres = set()
for genres_list in df2['movie_genres_list']:
    unique_genres.update(genres_list)

# Convertir la lista de géneros en variables dummy y agregarlas al DataFrame original
for genre in unique_genres:
    df2[genre] = df2['movie_genres_list'].apply(lambda x: 1 if genre in x else 0)

# Eliminar la columna temporal 'movie_genres_list'
df2.drop(columns=['movie_genres_list'], inplace=True)

# Mostrar las primeras filas del DataFrame resultante
df2.head()

,user_id,movie_id,rating,movie_title,movie_genres,movie_clean_title,movie_year,fecha_nueva,Adventure,IMAX,Children,Fantasy,Romance,Animation,Film-Noir,Horror,Drama,Crime,Western,Action,Mystery,(no genres listed),Thriller,Sci-Fi,Musical,Comedy,Documentary,War
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995),1995.0,2000-07-30,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995),1995.0,2000-07-30,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,Heat (1995),1995.0,2000-07-30,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven (a.k.a. Se7en) (1995),1995.0,2000-07-30,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The (1995)",1995.0,2000-07-30,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0


In [6]:
# Eliminar las columnas del DataFrame
peliculas = df2.drop(columns=['user_id', 'movie_id', 'rating', 'movie_title', 'movie_genres', 'fecha_nueva']).copy()
peliculas.reset_index(drop=True, inplace=True)
peliculas['movie_year'] = peliculas['movie_year'].fillna(0).astype(int)
# Eliminar duplicados basados en la columna 'movie_clean_title'
peliculas = peliculas.drop_duplicates(subset=['movie_clean_title'])
# escalar el año de publicacion
sc = MinMaxScaler()
peliculas.loc[:, "movie_year"] = sc.fit_transform(peliculas[['movie_year']])
# Reiniciar el índice
peliculas.reset_index(drop=True, inplace=True)

peliculas

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_16400\3732138851.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.98860258 0.98860258 0.98860258 ... 0.98860258 0.98463826 0.98860258]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  peliculas.loc[:, "movie_year"] = sc.fit_transform(peliculas[['movie_year']])


,movie_clean_title,movie_year,Adventure,IMAX,Children,Fantasy,Romance,Animation,Film-Noir,Horror,Drama,Crime,Western,Action,Mystery,(no genres listed),Thriller,Sci-Fi,Musical,Comedy,Documentary,War
0,Toy Story (1995),0.988603,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Grumpier Old Men (1995),0.988603,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Heat (1995),0.988603,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
3,Seven (a.k.a. Se7en) (1995),0.988603,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,"Usual Suspects, The (1995)",0.988603,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,Cirque du Freak: The Vampire's Assistant (2009),0.995540,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
7168,Alice (2009),0.995540,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7169,Fluke (1995),0.988603,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7170,Assassination (1987),0.984638,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0


In [7]:
# dummies
dummies = peliculas.drop(columns=['movie_clean_title'])
dummies

,movie_year,Adventure,IMAX,Children,Fantasy,Romance,Animation,Film-Noir,Horror,Drama,Crime,Western,Action,Mystery,(no genres listed),Thriller,Sci-Fi,Musical,Comedy,Documentary,War
0,0.988603,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0.988603,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.988603,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
3,0.988603,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,0.988603,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,0.995540,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
7168,0.995540,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7169,0.988603,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7170,0.984638,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0


In [8]:
##### ### entrenar modelo #####
## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar)
model = neighbors.NearestNeighbors(n_neighbors = 6, metric='cosine')
model.fit(dummies)
dist, idlist = model.kneighbors(dummies)
distancias = pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila (pelicula)
id_list = pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde


In [9]:
def MovieRecommender(movie_name=list(peliculas['movie_clean_title'].value_counts().index)):
    pelicula_list_name = []
    pelicula_id = peliculas[peliculas['movie_clean_title'] == movie_name].index
    pelicula_id = pelicula_id[0]
    # Obtener los vecinos más cercanos excluyendo la propia película
    neighbor_ids = [newid for newid in idlist[pelicula_id] if newid != pelicula_id]
    # Obtener los nombres de las películas vecinas
    for newid in neighbor_ids:
        pelicula_list_name.append(peliculas.loc[newid].movie_clean_title)
    recomendaciones = pd.DataFrame(pelicula_list_name, columns=['Peliculas Recomendadas'])
    return recomendaciones

print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Toy Story (1995)', "They Shoot Horses, Don'…

<function MovieRecommender at 0x000001BAEE3CF9C0>
